## Overview

This document summarizes the 2016 BY performance for the TRMG2 compared to the TRMv6.2


In [43]:
import pandas as pd
from siuba import *
import numpy as np
from plotnine import *


pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.float_format = '{:,.2f}'.format
set low_memory=False.





In [44]:
hwy_2016_g2 =  pd.read_csv('TRMG2_2016.csv')
hwy_2016_v6 =  pd.read_csv('TRMv6_2016.csv')

count =  pd.read_csv('TRM6_Count_2016_Observed.csv')



temp = (
count 
    >>select(_.ID, _.Daily_Counts)
)

temp = pd.merge(hwy_2016_v6, temp)

hwy_2016_v6_counts = (
temp
    >>filter(_.Daily_Counts > 0)
)



C:\Users\JacobFo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning:

Columns (49,50,51,55,56,57,59) have mixed types.Specify dtype option on import or set low_memory=False.

C:\Users\JacobFo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning:

Columns (20,21,25,26) have mixed types.Specify dtype option on import or set low_memory=False.



In [45]:
hwy_2016_g2.pivot_table(values="Total_Flow_Daily", index="MPO",  aggfunc=[np.sum, np.mean],fill_value=0)

,sum,mean
,Total_Flow_Daily,Total_Flow_Daily
MPO,,
CAMPO,"87,767,380.48","13,249.91"
DCHC,"44,499,404.10","12,282.47"
None,"8,027,109.89","4,769.52"


## Total Daily Flow by MPO: All Links

In [46]:
#hide_input
hwy_2016_g2.pivot_table(values="Total_Flow_Daily", index="MPO",  aggfunc=[np.sum, np.mean],fill_value=0)

,sum,mean
,Total_Flow_Daily,Total_Flow_Daily
MPO,,
CAMPO,"87,767,380.48","13,249.91"
DCHC,"44,499,404.10","12,282.47"
None,"8,027,109.89","4,769.52"
